In [1]:
import numpy as np

In [8]:
np.linspace(80, 120, 8 + 1)

array([ 80.,  85.,  90.,  95., 100., 105., 110., 115., 120.])

In [15]:
np.linspace(50, 150, 50+1)

array([ 50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,
        72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,  88.,  90.,  92.,
        94.,  96.,  98., 100., 102., 104., 106., 108., 110., 112., 114.,
       116., 118., 120., 122., 124., 126., 128., 130., 132., 134., 136.,
       138., 140., 142., 144., 146., 148., 150.])

In [16]:
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed

In [17]:
n_cpu_cores = multiprocessing.cpu_count()  # get the number of CPU cores
max_threads_cpu_task = n_cpu_cores - 2  # leave 2 cores free for other tasks

In [18]:
max_threads_cpu_task

18

In [12]:
from itertools import product

In [21]:
Option_types=["call", "put"]  # call, put
S_values=list(np.linspace(50, 150, 50 + 1))  # 50, 52, 54, ..., 150
K_values=[100]  # 100
T_values=[
    0.08,
    0.25,
    0.5,
    1.0,
    1.5,
    2.0,
    2.5,
    3.0,
    4.0,
    5.0,
]  # 1 month, 3 months, 6 months, 1 year, 1.5 years, 2 years, 2.5 years, 3 years, 4 years, 5 years
r_values=[0.01, 0.025, 0.05, 0.075, 0.1]  # 1%, 2.5%, 5%, 7.5%, 10%
sigma_values=[0.1, 0.2, 0.4, 0.6, 0.8]
q_values=[0.0, 0.01, 0.02, 0.03, 0.04, 0.05]

In [30]:
from typing import Literal
import numpy as np
import pandas as pd


def mc_price_worst_off(
    option_type: Literal["call", "put"],
    S1: float,
    S2: float,
    K1: float,
    K2: float,
    sigma1: float,
    sigma2: float,
    q1: float = 0.0,
    q2: float = 0.0,
    corr: float = 0.0,
    T: float = 1.0,
    r: float = 0.05,
    n_paths: int = 10000,  # number of paths to simulate, to make error 1/sqrt(n_paths) = 0.01 or 1%
) -> float:
    dt = T
    drift1 = dt * (r - q1 - 0.5 * sigma1**2)
    diffusion1 = sigma1 * np.sqrt(dt)

    Z1 = np.random.standard_normal(n_paths)
    ST1 = S1 * np.exp(drift1 + diffusion1 * Z1)

    dt = T
    drift2 = dt * (r - q2 - 0.5 * sigma2**2)
    diffusion2 = sigma2 * np.sqrt(dt)

    Z2 = np.random.standard_normal(n_paths)
    ST2 = S2 * np.exp(drift2 + diffusion2 * (corr * Z1 + np.sqrt(1 - corr**2) * Z2))

    if option_type == "call":
        # Confusion with what said in the word document in the email
        # performance_of_first = np.maximum(ST1/S1, 0.0)
        # performance_of_second = np.maximum(ST2/S2, 0.0)

        performance_of_first = np.maximum(ST1 - K1, 0.0)
        performance_of_second = np.maximum(ST2 - K2, 0.0)
        payoffs = np.minimum(performance_of_first, performance_of_second)
    else:
        performance_of_first = np.maximum(K1 - ST1, 0.0)
        performance_of_second = np.maximum(K2 - ST2, 0.0)
        payoffs = np.minimum(performance_of_first, performance_of_second)

    res = {
        "S1" : [S1]*10000,
        "ST1" : ST1,
        "P1" : performance_of_first,
        "S2" : [S2]*10000,
        "ST2" : ST2,
        "P2" : performance_of_second,
        "payoff" : payoffs
    }
    res = pd.DataFrame(res)
    
    print("shape of worst off", payoffs.shape)
    price = np.exp(-r * T) * payoffs.mean()  # discount factor for risk-free rate
    return price, res

In [33]:
price , res = mc_price_worst_off(
    "call",
    np.float64(100.0),
    np.float64(100.0),
    100,
    100,
    0.1,
    0.1,
    0.01,
    0.01,
    0.01,
    0.08,
    0.01,
)

shape of worst off (10000,)


In [35]:
res

,S1,ST1,P1,S2,ST2,P2,payoff
0,100.0,99.814481,0.000000,100.0,100.714262,0.714262,0.000000
1,100.0,105.041496,5.041496,100.0,97.827702,0.000000,0.000000
2,100.0,96.860109,0.000000,100.0,97.875926,0.000000,0.000000
3,100.0,98.561025,0.000000,100.0,101.670757,1.670757,0.000000
4,100.0,100.396476,0.396476,100.0,103.009539,3.009539,0.396476
...,...,...,...,...,...,...,...
9995,100.0,97.154849,0.000000,100.0,104.969948,4.969948,0.000000
9996,100.0,97.378225,0.000000,100.0,100.854286,0.854286,0.000000
9997,100.0,101.107652,1.107652,100.0,103.615702,3.615702,1.107652
9998,100.0,100.545155,0.545155,100.0,98.496259,0.000000,0.000000


In [20]:
Option_types=["call", "put"]
S1_values=list(np.linspace(80, 120, 8 + 1))
S2_values=list(np.linspace(80, 120, 8 + 1))
K1_values=[100]
K2_values=[100]
sigma1_values=[0.1, 0.4, 0.8]
sigma2_values=[0.1, 0.4, 0.8]
q1_values=[0.01, 0.02, 0.05]
q2_values=[0.01, 0.02, 0.05]
corr_values=[0.01, 0.2, 0.5, 0.8]
T_values=[0.08, 0.5, 1.0, 2.0, 5.0]
r_values = [0.01, 0.025, 0.05, 0.1]

In [22]:
for _ in product(
        Option_types,
        S1_values,
        S2_values,
        K1_values,
        K2_values,
        sigma1_values,
        sigma2_values,
        q1_values,
        q2_values,
        corr_values,
        T_values,
        r_values,
    ):
    print(_)
    break

('call', np.float64(80.0), np.float64(80.0), 100, 100, 0.1, 0.1, 0.01, 0.01, 0.01, 0.08, 0.01)


In [21]:
n = sum(
    1
    for _ in product(
        Option_types,
        S1_values,
        S2_values,
        K1_values,
        K2_values,
        sigma1_values,
        sigma2_values,
        q1_values,
        q2_values,
        corr_values,
        T_values,
        r_values,
    )
)
print("Number of combinations:", n)

Number of combinations: 1049760


In [23]:
import math

lengths = [
    len(Option_types),
    len(S_values),
    len(K_values),
    len(T_values),
    len(r_values),
    len(sigma_values),
    len(q_values),
]

n_combinations = math.prod(lengths)
print("Number of combinations:", n_combinations)

Number of combinations: 153000


In [ ]:
out_features = [
    "stock_price_1",
    "stock_price_2",
    "strike_price_1",
    "strike_price_2",
    "volatility_1",
    "volatility_2",
    "dividend_yield_1",
    "dividend_yield_2",
    "correlation",
    "time_to_maturity",
    "interest_rate",
    "1_over_T",
    "log_T",
    "sqrt_T",
    "log1p_T",
    "variance_1",
    "variance_2",
    "stock_1_over_strike_1",
    "stock_2_over_strike_2",
    "strike_1_over_stock_1",
    "strike_2_over_stock_2",
    "volatility_1_over_interest",
    "volatility_2_over_interest",
    "dividend_yield_1_over_interest",
    "dividend_yield_2_over_interest",
    "stock_price_1_over_interest",
    "stock_price_2_over_interest",
    "volatility_1_over_dividend_1",
    "interest_rate_over_dividend_1",
    "stock_price_1_over_dividend_1",
    "volatility_2_over_dividend_2",
    "interest_rate_over_dividend_2",
    "stock_price_2_over_dividend_2",
    "interest_rate_over_volatility_1",
    "dividend_yield_1_over_volatility_1",
    "stock_price_1_over_volatility_1",
    "interest_rate_over_volatility_2",
    "dividend_yield_2_over_volatility_2",
    "stock_price_2_over_volatility_2",
    "stock_price_1_times_stock_price_2",
    "stock_price_1_times_strike_price_1",
    "stock_price_1_times_strike_price_2",
    "stock_price_1_times_volatility_1",
    "stock_price_1_times_volatility_2",
    "stock_price_1_times_dividend_yield_1",
    "stock_price_1_times_dividend_yield_2",
    "stock_price_1_times_correlation",
    "stock_price_1_times_time_to_maturity",
    "stock_price_1_times_interest_rate",
    "stock_price_1_times_1_over_T",
    "stock_price_1_times_log_T",
    "stock_price_1_times_sqrt_T",
    "stock_price_1_times_log1p_T",
    "stock_price_1_times_variance_1",
    "stock_price_1_times_variance_2",
    "stock_price_1_times_stock_1_over_strike_1",
    "stock_price_1_times_stock_2_over_strike_2",
    "stock_price_1_times_strike_1_over_stock_1",
    "stock_price_1_times_strike_2_over_stock_2",
    "stock_price_1_times_volatility_1_over_interest",
    "stock_price_1_times_volatility_2_over_interest",
    "stock_price_1_times_dividend_yield_1_over_interest",
    "stock_price_1_times_dividend_yield_2_over_interest",
    "stock_price_1_times_stock_price_1_over_interest",
    "stock_price_1_times_stock_price_2_over_interest",
    "stock_price_1_times_volatility_1_over_dividend_1",
    "stock_price_1_times_interest_rate_over_dividend_1",
    "stock_price_1_times_stock_price_1_over_dividend_1",
    "stock_price_1_times_volatility_2_over_dividend_2",
    "stock_price_1_times_interest_rate_over_dividend_2",
    "stock_price_1_times_stock_price_2_over_dividend_2",
    "stock_price_1_times_interest_rate_over_volatility_1",
    "stock_price_1_times_dividend_yield_1_over_volatility_1",
    "stock_price_1_times_stock_price_1_over_volatility_1",
    "stock_price_1_times_interest_rate_over_volatility_2",
    "stock_price_1_times_dividend_yield_2_over_volatility_2",
    "stock_price_1_times_stock_price_2_over_volatility_2",
    "stock_price_2_times_strike_price_1",
    "stock_price_2_times_strike_price_2",
    "stock_price_2_times_volatility_1",
    "stock_price_2_times_volatility_2",
    "stock_price_2_times_dividend_yield_1",
    "stock_price_2_times_dividend_yield_2",
    "stock_price_2_times_correlation",
    "stock_price_2_times_time_to_maturity",
    "stock_price_2_times_interest_rate",
    "stock_price_2_times_1_over_T",
    "stock_price_2_times_log_T",
    "stock_price_2_times_sqrt_T",
    "stock_price_2_times_log1p_T",
    "stock_price_2_times_variance_1",
    "stock_price_2_times_variance_2",
    "stock_price_2_times_stock_1_over_strike_1",
    "stock_price_2_times_stock_2_over_strike_2",
    "stock_price_2_times_strike_1_over_stock_1",
    "stock_price_2_times_strike_2_over_stock_2",
    "stock_price_2_times_volatility_1_over_interest",
    "stock_price_2_times_volatility_2_over_interest",
    "stock_price_2_times_dividend_yield_1_over_interest",
    "stock_price_2_times_dividend_yield_2_over_interest",
    "stock_price_2_times_stock_price_1_over_interest",
    "stock_price_2_times_stock_price_2_over_interest",
    "stock_price_2_times_volatility_1_over_dividend_1",
    "stock_price_2_times_interest_rate_over_dividend_1",
    "stock_price_2_times_stock_price_1_over_dividend_1",
    "stock_price_2_times_volatility_2_over_dividend_2",
    "stock_price_2_times_interest_rate_over_dividend_2",
    "stock_price_2_times_stock_price_2_over_dividend_2",
    "stock_price_2_times_interest_rate_over_volatility_1",
    "stock_price_2_times_dividend_yield_1_over_volatility_1",
    "stock_price_2_times_stock_price_1_over_volatility_1",
    "stock_price_2_times_interest_rate_over_volatility_2",
    "stock_price_2_times_dividend_yield_2_over_volatility_2",
    "stock_price_2_times_stock_price_2_over_volatility_2",
    "strike_price_1_times_strike_price_2",
    "strike_price_1_times_volatility_1",
    "strike_price_1_times_volatility_2",
    "strike_price_1_times_dividend_yield_1",
    "strike_price_1_times_dividend_yield_2",
    "strike_price_1_times_correlation",
    "strike_price_1_times_time_to_maturity",
    "strike_price_1_times_interest_rate",
    "strike_price_1_times_1_over_T",
    "strike_price_1_times_log_T",
    "strike_price_1_times_sqrt_T",
    "strike_price_1_times_log1p_T",
    "strike_price_1_times_variance_1",
    "strike_price_1_times_variance_2",
    "strike_price_1_times_stock_1_over_strike_1",
    "strike_price_1_times_stock_2_over_strike_2",
    "strike_price_1_times_strike_1_over_stock_1",
    "strike_price_1_times_strike_2_over_stock_2",
    "strike_price_1_times_volatility_1_over_interest",
    "strike_price_1_times_volatility_2_over_interest",
    "strike_price_1_times_dividend_yield_1_over_interest",
    "strike_price_1_times_dividend_yield_2_over_interest",
    "strike_price_1_times_stock_price_1_over_interest",
    "strike_price_1_times_stock_price_2_over_interest",
    "strike_price_1_times_volatility_1_over_dividend_1",
    "strike_price_1_times_interest_rate_over_dividend_1",
    "strike_price_1_times_stock_price_1_over_dividend_1",
    "strike_price_1_times_volatility_2_over_dividend_2",
    "strike_price_1_times_interest_rate_over_dividend_2",
    "strike_price_1_times_stock_price_2_over_dividend_2",
    "strike_price_1_times_interest_rate_over_volatility_1",
    "strike_price_1_times_dividend_yield_1_over_volatility_1",
    "strike_price_1_times_stock_price_1_over_volatility_1",
    "strike_price_1_times_interest_rate_over_volatility_2",
    "strike_price_1_times_dividend_yield_2_over_volatility_2",
    "strike_price_1_times_stock_price_2_over_volatility_2",
    "strike_price_2_times_volatility_1",
    "strike_price_2_times_volatility_2",
    "strike_price_2_times_dividend_yield_1",
    "strike_price_2_times_dividend_yield_2",
    "strike_price_2_times_correlation",
    "strike_price_2_times_time_to_maturity",
    "strike_price_2_times_interest_rate",
    "strike_price_2_times_1_over_T",
    "strike_price_2_times_log_T",
    "strike_price_2_times_sqrt_T",
    "strike_price_2_times_log1p_T",
    "strike_price_2_times_variance_1",
    "strike_price_2_times_variance_2",
    "strike_price_2_times_stock_1_over_strike_1",
    "strike_price_2_times_stock_2_over_strike_2",
    "strike_price_2_times_strike_1_over_stock_1",
    "strike_price_2_times_strike_2_over_stock_2",
    "strike_price_2_times_volatility_1_over_interest",
    "strike_price_2_times_volatility_2_over_interest",
    "strike_price_2_times_dividend_yield_1_over_interest",
    "strike_price_2_times_dividend_yield_2_over_interest",
    "strike_price_2_times_stock_price_1_over_interest",
    "strike_price_2_times_stock_price_2_over_interest",
    "strike_price_2_times_volatility_1_over_dividend_1",
    "strike_price_2_times_interest_rate_over_dividend_1",
    "strike_price_2_times_stock_price_1_over_dividend_1",
    "strike_price_2_times_volatility_2_over_dividend_2",
    "strike_price_2_times_interest_rate_over_dividend_2",
    "strike_price_2_times_stock_price_2_over_dividend_2",
    "strike_price_2_times_interest_rate_over_volatility_1",
    "strike_price_2_times_dividend_yield_1_over_volatility_1",
    "strike_price_2_times_stock_price_1_over_volatility_1",
    "strike_price_2_times_interest_rate_over_volatility_2",
    "strike_price_2_times_dividend_yield_2_over_volatility_2",
    "strike_price_2_times_stock_price_2_over_volatility_2",
    "volatility_1_times_volatility_2",
    "volatility_1_times_dividend_yield_1",
    "volatility_1_times_dividend_yield_2",
    "volatility_1_times_correlation",
    "volatility_1_times_time_to_maturity",
    "volatility_1_times_interest_rate",
    "volatility_1_times_1_over_T",
    "volatility_1_times_log_T",
    "volatility_1_times_sqrt_T",
    "volatility_1_times_log1p_T",
    "volatility_1_times_variance_1",
    "volatility_1_times_variance_2",
    "volatility_1_times_stock_1_over_strike_1",
    "volatility_1_times_stock_2_over_strike_2",
    "volatility_1_times_strike_1_over_stock_1",
    "volatility_1_times_strike_2_over_stock_2",
    "volatility_1_times_volatility_1_over_interest",
    "volatility_1_times_volatility_2_over_interest",
    "volatility_1_times_dividend_yield_1_over_interest",
    "volatility_1_times_dividend_yield_2_over_interest",
    "volatility_1_times_stock_price_1_over_interest",
    "volatility_1_times_stock_price_2_over_interest",
    "volatility_1_times_volatility_1_over_dividend_1",
    "volatility_1_times_interest_rate_over_dividend_1",
    "volatility_1_times_stock_price_1_over_dividend_1",
    "volatility_1_times_volatility_2_over_dividend_2",
    "volatility_1_times_interest_rate_over_dividend_2",
    "volatility_1_times_stock_price_2_over_dividend_2",
    "volatility_1_times_interest_rate_over_volatility_1",
    "volatility_1_times_dividend_yield_1_over_volatility_1",
    "volatility_1_times_stock_price_1_over_volatility_1",
    "volatility_1_times_interest_rate_over_volatility_2",
    "volatility_1_times_dividend_yield_2_over_volatility_2",
    "volatility_1_times_stock_price_2_over_volatility_2",
    "volatility_2_times_dividend_yield_1",
    "volatility_2_times_dividend_yield_2",
    "volatility_2_times_correlation",
    "volatility_2_times_time_to_maturity",
    "volatility_2_times_interest_rate",
    "volatility_2_times_1_over_T",
    "volatility_2_times_log_T",
    "volatility_2_times_sqrt_T",
    "volatility_2_times_log1p_T",
    "volatility_2_times_variance_1",
    "volatility_2_times_variance_2",
    "volatility_2_times_stock_1_over_strike_1",
    "volatility_2_times_stock_2_over_strike_2",
    "volatility_2_times_strike_1_over_stock_1",
    "volatility_2_times_strike_2_over_stock_2",
    "volatility_2_times_volatility_1_over_interest",
    "volatility_2_times_volatility_2_over_interest",
    "volatility_2_times_dividend_yield_1_over_interest",
    "volatility_2_times_dividend_yield_2_over_interest",
    "volatility_2_times_stock_price_1_over_interest",
    "volatility_2_times_stock_price_2_over_interest",
    "volatility_2_times_volatility_1_over_dividend_1",
    "volatility_2_times_interest_rate_over_dividend_1",
    "volatility_2_times_stock_price_1_over_dividend_1",
    "volatility_2_times_volatility_2_over_dividend_2",
    "volatility_2_times_interest_rate_over_dividend_2",
    "volatility_2_times_stock_price_2_over_dividend_2",
    "volatility_2_times_interest_rate_over_volatility_1",
    "volatility_2_times_dividend_yield_1_over_volatility_1",
    "volatility_2_times_stock_price_1_over_volatility_1",
    "volatility_2_times_interest_rate_over_volatility_2",
    "volatility_2_times_dividend_yield_2_over_volatility_2",
    "volatility_2_times_stock_price_2_over_volatility_2",
    "dividend_yield_1_times_dividend_yield_2",
    "dividend_yield_1_times_correlation",
    "dividend_yield_1_times_time_to_maturity",
    "dividend_yield_1_times_interest_rate",
    "dividend_yield_1_times_1_over_T",
    "dividend_yield_1_times_log_T",
    "dividend_yield_1_times_sqrt_T",
    "dividend_yield_1_times_log1p_T",
    "dividend_yield_1_times_variance_1",
    "dividend_yield_1_times_variance_2",
    "dividend_yield_1_times_stock_1_over_strike_1",
    "dividend_yield_1_times_stock_2_over_strike_2",
    "dividend_yield_1_times_strike_1_over_stock_1",
    "dividend_yield_1_times_strike_2_over_stock_2",
    "dividend_yield_1_times_volatility_1_over_interest",
    "dividend_yield_1_times_volatility_2_over_interest",
    "dividend_yield_1_times_dividend_yield_1_over_interest",
    "dividend_yield_1_times_dividend_yield_2_over_interest",
    "dividend_yield_1_times_stock_price_1_over_interest",
    "dividend_yield_1_times_stock_price_2_over_interest",
    "dividend_yield_1_times_volatility_1_over_dividend_1",
    "dividend_yield_1_times_interest_rate_over_dividend_1",
    "dividend_yield_1_times_stock_price_1_over_dividend_1",
    "dividend_yield_1_times_volatility_2_over_dividend_2",
    "dividend_yield_1_times_interest_rate_over_dividend_2",
    "dividend_yield_1_times_stock_price_2_over_dividend_2",
    "dividend_yield_1_times_interest_rate_over_volatility_1",
    "dividend_yield_1_times_dividend_yield_1_over_volatility_1",
    "dividend_yield_1_times_stock_price_1_over_volatility_1",
    "dividend_yield_1_times_interest_rate_over_volatility_2",
    "dividend_yield_1_times_dividend_yield_2_over_volatility_2",
    "dividend_yield_1_times_stock_price_2_over_volatility_2",
    "dividend_yield_2_times_correlation",
    "dividend_yield_2_times_time_to_maturity",
    "dividend_yield_2_times_interest_rate",
    "dividend_yield_2_times_1_over_T",
    "dividend_yield_2_times_log_T",
    "dividend_yield_2_times_sqrt_T",
    "dividend_yield_2_times_log1p_T",
    "dividend_yield_2_times_variance_1",
    "dividend_yield_2_times_variance_2",
    "dividend_yield_2_times_stock_1_over_strike_1",
    "dividend_yield_2_times_stock_2_over_strike_2",
    "dividend_yield_2_times_strike_1_over_stock_1",
    "dividend_yield_2_times_strike_2_over_stock_2",
    "dividend_yield_2_times_volatility_1_over_interest",
    "dividend_yield_2_times_volatility_2_over_interest",
    "dividend_yield_2_times_dividend_yield_1_over_interest",
    "dividend_yield_2_times_dividend_yield_2_over_interest",
    "dividend_yield_2_times_stock_price_1_over_interest",
    "dividend_yield_2_times_stock_price_2_over_interest",
    "dividend_yield_2_times_volatility_1_over_dividend_1",
    "dividend_yield_2_times_interest_rate_over_dividend_1",
    "dividend_yield_2_times_stock_price_1_over_dividend_1",
    "dividend_yield_2_times_volatility_2_over_dividend_2",
    "dividend_yield_2_times_interest_rate_over_dividend_2",
    "dividend_yield_2_times_stock_price_2_over_dividend_2",
    "dividend_yield_2_times_interest_rate_over_volatility_1",
    "dividend_yield_2_times_dividend_yield_1_over_volatility_1",
    "dividend_yield_2_times_stock_price_1_over_volatility_1",
    "dividend_yield_2_times_interest_rate_over_volatility_2",
    "dividend_yield_2_times_dividend_yield_2_over_volatility_2",
    "dividend_yield_2_times_stock_price_2_over_volatility_2",
    "correlation_times_time_to_maturity",
    "correlation_times_interest_rate",
    "correlation_times_1_over_T",
    "correlation_times_log_T",
    "correlation_times_sqrt_T",
    "correlation_times_log1p_T",
    "correlation_times_variance_1",
    "correlation_times_variance_2",
    "correlation_times_stock_1_over_strike_1",
    "correlation_times_stock_2_over_strike_2",
    "correlation_times_strike_1_over_stock_1",
    "correlation_times_strike_2_over_stock_2",
    "correlation_times_volatility_1_over_interest",
    "correlation_times_volatility_2_over_interest",
    "correlation_times_dividend_yield_1_over_interest",
    "correlation_times_dividend_yield_2_over_interest",
    "correlation_times_stock_price_1_over_interest",
    "correlation_times_stock_price_2_over_interest",
    "correlation_times_volatility_1_over_dividend_1",
    "correlation_times_interest_rate_over_dividend_1",
    "correlation_times_stock_price_1_over_dividend_1",
    "correlation_times_volatility_2_over_dividend_2",
    "correlation_times_interest_rate_over_dividend_2",
    "correlation_times_stock_price_2_over_dividend_2",
    "correlation_times_interest_rate_over_volatility_1",
    "correlation_times_dividend_yield_1_over_volatility_1",
    "correlation_times_stock_price_1_over_volatility_1",
    "correlation_times_interest_rate_over_volatility_2",
    "correlation_times_dividend_yield_2_over_volatility_2",
    "correlation_times_stock_price_2_over_volatility_2",
    "time_to_maturity_times_interest_rate",
    "time_to_maturity_times_1_over_T",
    "time_to_maturity_times_log_T",
    "time_to_maturity_times_sqrt_T",
    "time_to_maturity_times_log1p_T",
    "time_to_maturity_times_variance_1",
    "time_to_maturity_times_variance_2",
    "time_to_maturity_times_stock_1_over_strike_1",
    "time_to_maturity_times_stock_2_over_strike_2",
    "time_to_maturity_times_strike_1_over_stock_1",
    "time_to_maturity_times_strike_2_over_stock_2",
    "time_to_maturity_times_volatility_1_over_interest",
    "time_to_maturity_times_volatility_2_over_interest",
    "time_to_maturity_times_dividend_yield_1_over_interest",
    "time_to_maturity_times_dividend_yield_2_over_interest",
    "time_to_maturity_times_stock_price_1_over_interest",
    "time_to_maturity_times_stock_price_2_over_interest",
    "time_to_maturity_times_volatility_1_over_dividend_1",
    "time_to_maturity_times_interest_rate_over_dividend_1",
    "time_to_maturity_times_stock_price_1_over_dividend_1",
    "time_to_maturity_times_volatility_2_over_dividend_2",
    "time_to_maturity_times_interest_rate_over_dividend_2",
    "time_to_maturity_times_stock_price_2_over_dividend_2",
    "time_to_maturity_times_interest_rate_over_volatility_1",
    "time_to_maturity_times_dividend_yield_1_over_volatility_1",
    "time_to_maturity_times_stock_price_1_over_volatility_1",
    "time_to_maturity_times_interest_rate_over_volatility_2",
    "time_to_maturity_times_dividend_yield_2_over_volatility_2",
    "time_to_maturity_times_stock_price_2_over_volatility_2",
    "interest_rate_times_1_over_T",
    "interest_rate_times_log_T",
    "interest_rate_times_sqrt_T",
    "interest_rate_times_log1p_T",
    "interest_rate_times_variance_1",
    "interest_rate_times_variance_2",
    "interest_rate_times_stock_1_over_strike_1",
    "interest_rate_times_stock_2_over_strike_2",
    "interest_rate_times_strike_1_over_stock_1",
    "interest_rate_times_strike_2_over_stock_2",
    "interest_rate_times_volatility_1_over_interest",
    "interest_rate_times_volatility_2_over_interest",
    "interest_rate_times_dividend_yield_1_over_interest",
    "interest_rate_times_dividend_yield_2_over_interest",
    "interest_rate_times_stock_price_1_over_interest",
    "interest_rate_times_stock_price_2_over_interest",
    "interest_rate_times_volatility_1_over_dividend_1",
    "interest_rate_times_interest_rate_over_dividend_1",
    "interest_rate_times_stock_price_1_over_dividend_1",
    "interest_rate_times_volatility_2_over_dividend_2",
    "interest_rate_times_interest_rate_over_dividend_2",
    "interest_rate_times_stock_price_2_over_dividend_2",
    "interest_rate_times_interest_rate_over_volatility_1",
    "interest_rate_times_dividend_yield_1_over_volatility_1",
    "interest_rate_times_stock_price_1_over_volatility_1",
    "interest_rate_times_interest_rate_over_volatility_2",
    "interest_rate_times_dividend_yield_2_over_volatility_2",
    "interest_rate_times_stock_price_2_over_volatility_2",
    "1_over_T_times_log_T",
    "1_over_T_times_sqrt_T",
    "1_over_T_times_log1p_T",
    "1_over_T_times_variance_1",
    "1_over_T_times_variance_2",
    "1_over_T_times_stock_1_over_strike_1",
    "1_over_T_times_stock_2_over_strike_2",
    "1_over_T_times_strike_1_over_stock_1",
    "1_over_T_times_strike_2_over_stock_2",
    "1_over_T_times_volatility_1_over_interest",
    "1_over_T_times_volatility_2_over_interest",
    "1_over_T_times_dividend_yield_1_over_interest",
    "1_over_T_times_dividend_yield_2_over_interest",
    "1_over_T_times_stock_price_1_over_interest",
    "1_over_T_times_stock_price_2_over_interest",
    "1_over_T_times_volatility_1_over_dividend_1",
    "1_over_T_times_interest_rate_over_dividend_1",
    "1_over_T_times_stock_price_1_over_dividend_1",
    "1_over_T_times_volatility_2_over_dividend_2",
    "1_over_T_times_interest_rate_over_dividend_2",
    "1_over_T_times_stock_price_2_over_dividend_2",
    "1_over_T_times_interest_rate_over_volatility_1",
    "1_over_T_times_dividend_yield_1_over_volatility_1",
    "1_over_T_times_stock_price_1_over_volatility_1",
    "1_over_T_times_interest_rate_over_volatility_2",
    "1_over_T_times_dividend_yield_2_over_volatility_2",
    "1_over_T_times_stock_price_2_over_volatility_2",
    "log_T_times_sqrt_T",
    "log_T_times_log1p_T",
    "log_T_times_variance_1",
    "log_T_times_variance_2",
    "log_T_times_stock_1_over_strike_1",
    "log_T_times_stock_2_over_strike_2",
    "log_T_times_strike_1_over_stock_1",
    "log_T_times_strike_2_over_stock_2",
    "log_T_times_volatility_1_over_interest",
    "log_T_times_volatility_2_over_interest",
    "log_T_times_dividend_yield_1_over_interest",
    "log_T_times_dividend_yield_2_over_interest",
    "log_T_times_stock_price_1_over_interest",
    "log_T_times_stock_price_2_over_interest",
    "log_T_times_volatility_1_over_dividend_1",
    "log_T_times_interest_rate_over_dividend_1",
    "log_T_times_stock_price_1_over_dividend_1",
    "log_T_times_volatility_2_over_dividend_2",
    "log_T_times_interest_rate_over_dividend_2",
    "log_T_times_stock_price_2_over_dividend_2",
    "log_T_times_interest_rate_over_volatility_1",
    "log_T_times_dividend_yield_1_over_volatility_1",
    "log_T_times_stock_price_1_over_volatility_1",
    "log_T_times_interest_rate_over_volatility_2",
    "log_T_times_dividend_yield_2_over_volatility_2",
    "log_T_times_stock_price_2_over_volatility_2",
    "sqrt_T_times_log1p_T",
    "sqrt_T_times_variance_1",
    "sqrt_T_times_variance_2",
    "sqrt_T_times_stock_1_over_strike_1",
    "sqrt_T_times_stock_2_over_strike_2",
    "sqrt_T_times_strike_1_over_stock_1",
    "sqrt_T_times_strike_2_over_stock_2",
    "sqrt_T_times_volatility_1_over_interest",
    "sqrt_T_times_volatility_2_over_interest",
    "sqrt_T_times_dividend_yield_1_over_interest",
    "sqrt_T_times_dividend_yield_2_over_interest",
    "sqrt_T_times_stock_price_1_over_interest",
    "sqrt_T_times_stock_price_2_over_interest",
    "sqrt_T_times_volatility_1_over_dividend_1",
    "sqrt_T_times_interest_rate_over_dividend_1",
    "sqrt_T_times_stock_price_1_over_dividend_1",
    "sqrt_T_times_volatility_2_over_dividend_2",
    "sqrt_T_times_interest_rate_over_dividend_2",
    "sqrt_T_times_stock_price_2_over_dividend_2",
    "sqrt_T_times_interest_rate_over_volatility_1",
    "sqrt_T_times_dividend_yield_1_over_volatility_1",
    "sqrt_T_times_stock_price_1_over_volatility_1",
    "sqrt_T_times_interest_rate_over_volatility_2",
    "sqrt_T_times_dividend_yield_2_over_volatility_2",
    "sqrt_T_times_stock_price_2_over_volatility_2",
    "log1p_T_times_variance_1",
    "log1p_T_times_variance_2",
    "log1p_T_times_stock_1_over_strike_1",
    "log1p_T_times_stock_2_over_strike_2",
    "log1p_T_times_strike_1_over_stock_1",
    "log1p_T_times_strike_2_over_stock_2",
    "log1p_T_times_volatility_1_over_interest",
    "log1p_T_times_volatility_2_over_interest",
    "log1p_T_times_dividend_yield_1_over_interest",
    "log1p_T_times_dividend_yield_2_over_interest",
    "log1p_T_times_stock_price_1_over_interest",
    "log1p_T_times_stock_price_2_over_interest",
    "log1p_T_times_volatility_1_over_dividend_1",
    "log1p_T_times_interest_rate_over_dividend_1",
    "log1p_T_times_stock_price_1_over_dividend_1",
    "log1p_T_times_volatility_2_over_dividend_2",
    "log1p_T_times_interest_rate_over_dividend_2",
    "log1p_T_times_stock_price_2_over_dividend_2",
    "log1p_T_times_interest_rate_over_volatility_1",
    "log1p_T_times_dividend_yield_1_over_volatility_1",
    "log1p_T_times_stock_price_1_over_volatility_1",
    "log1p_T_times_interest_rate_over_volatility_2",
    "log1p_T_times_dividend_yield_2_over_volatility_2",
    "log1p_T_times_stock_price_2_over_volatility_2",
    "variance_1_times_variance_2",
    "variance_1_times_stock_1_over_strike_1",
    "variance_1_times_stock_2_over_strike_2",
    "variance_1_times_strike_1_over_stock_1",
    "variance_1_times_strike_2_over_stock_2",
    "variance_1_times_volatility_1_over_interest",
    "variance_1_times_volatility_2_over_interest",
    "variance_1_times_dividend_yield_1_over_interest",
    "variance_1_times_dividend_yield_2_over_interest",
    "variance_1_times_stock_price_1_over_interest",
    "variance_1_times_stock_price_2_over_interest",
    "variance_1_times_volatility_1_over_dividend_1",
    "variance_1_times_interest_rate_over_dividend_1",
    "variance_1_times_stock_price_1_over_dividend_1",
    "variance_1_times_volatility_2_over_dividend_2",
    "variance_1_times_interest_rate_over_dividend_2",
    "variance_1_times_stock_price_2_over_dividend_2",
    "variance_1_times_interest_rate_over_volatility_1",
    "variance_1_times_dividend_yield_1_over_volatility_1",
    "variance_1_times_stock_price_1_over_volatility_1",
    "variance_1_times_interest_rate_over_volatility_2",
    "variance_1_times_dividend_yield_2_over_volatility_2",
    "variance_1_times_stock_price_2_over_volatility_2",
    "variance_2_times_stock_1_over_strike_1",
    "variance_2_times_stock_2_over_strike_2",
    "variance_2_times_strike_1_over_stock_1",
    "variance_2_times_strike_2_over_stock_2",
    "variance_2_times_volatility_1_over_interest",
    "variance_2_times_volatility_2_over_interest",
    "variance_2_times_dividend_yield_1_over_interest",
    "variance_2_times_dividend_yield_2_over_interest",
    "variance_2_times_stock_price_1_over_interest",
    "variance_2_times_stock_price_2_over_interest",
    "variance_2_times_volatility_1_over_dividend_1",
    "variance_2_times_interest_rate_over_dividend_1",
    "variance_2_times_stock_price_1_over_dividend_1",
    "variance_2_times_volatility_2_over_dividend_2",
    "variance_2_times_interest_rate_over_dividend_2",
    "variance_2_times_stock_price_2_over_dividend_2",
    "variance_2_times_interest_rate_over_volatility_1",
    "variance_2_times_dividend_yield_1_over_volatility_1",
    "variance_2_times_stock_price_1_over_volatility_1",
    "variance_2_times_interest_rate_over_volatility_2",
    "variance_2_times_dividend_yield_2_over_volatility_2",
    "variance_2_times_stock_price_2_over_volatility_2",
    "stock_1_over_strike_1_times_stock_2_over_strike_2",
    "stock_1_over_strike_1_times_strike_1_over_stock_1",
    "stock_1_over_strike_1_times_strike_2_over_stock_2",
    "stock_1_over_strike_1_times_volatility_1_over_interest",
    "stock_1_over_strike_1_times_volatility_2_over_interest",
    "stock_1_over_strike_1_times_dividend_yield_1_over_interest",
    "stock_1_over_strike_1_times_dividend_yield_2_over_interest",
    "stock_1_over_strike_1_times_stock_price_1_over_interest",
    "stock_1_over_strike_1_times_stock_price_2_over_interest",
    "stock_1_over_strike_1_times_volatility_1_over_dividend_1",
    "stock_1_over_strike_1_times_interest_rate_over_dividend_1",
    "stock_1_over_strike_1_times_stock_price_1_over_dividend_1",
    "stock_1_over_strike_1_times_volatility_2_over_dividend_2",
    "stock_1_over_strike_1_times_interest_rate_over_dividend_2",
    "stock_1_over_strike_1_times_stock_price_2_over_dividend_2",
    "stock_1_over_strike_1_times_interest_rate_over_volatility_1",
    "stock_1_over_strike_1_times_dividend_yield_1_over_volatility_1",
    "stock_1_over_strike_1_times_stock_price_1_over_volatility_1",
    "stock_1_over_strike_1_times_interest_rate_over_volatility_2",
    "stock_1_over_strike_1_times_dividend_yield_2_over_volatility_2",
    "stock_1_over_strike_1_times_stock_price_2_over_volatility_2",
    "stock_2_over_strike_2_times_strike_1_over_stock_1",
    "stock_2_over_strike_2_times_strike_2_over_stock_2",
    "stock_2_over_strike_2_times_volatility_1_over_interest",
    "stock_2_over_strike_2_times_volatility_2_over_interest",
    "stock_2_over_strike_2_times_dividend_yield_1_over_interest",
    "stock_2_over_strike_2_times_dividend_yield_2_over_interest",
    "stock_2_over_strike_2_times_stock_price_1_over_interest",
    "stock_2_over_strike_2_times_stock_price_2_over_interest",
    "stock_2_over_strike_2_times_volatility_1_over_dividend_1",
    "stock_2_over_strike_2_times_interest_rate_over_dividend_1",
    "stock_2_over_strike_2_times_stock_price_1_over_dividend_1",
    "stock_2_over_strike_2_times_volatility_2_over_dividend_2",
    "stock_2_over_strike_2_times_interest_rate_over_dividend_2",
    "stock_2_over_strike_2_times_stock_price_2_over_dividend_2",
    "stock_2_over_strike_2_times_interest_rate_over_volatility_1",
    "stock_2_over_strike_2_times_dividend_yield_1_over_volatility_1",
    "stock_2_over_strike_2_times_stock_price_1_over_volatility_1",
    "stock_2_over_strike_2_times_interest_rate_over_volatility_2",
    "stock_2_over_strike_2_times_dividend_yield_2_over_volatility_2",
    "stock_2_over_strike_2_times_stock_price_2_over_volatility_2",
    "strike_1_over_stock_1_times_strike_2_over_stock_2",
    "strike_1_over_stock_1_times_volatility_1_over_interest",
    "strike_1_over_stock_1_times_volatility_2_over_interest",
    "strike_1_over_stock_1_times_dividend_yield_1_over_interest",
    "strike_1_over_stock_1_times_dividend_yield_2_over_interest",
    "strike_1_over_stock_1_times_stock_price_1_over_interest",
    "strike_1_over_stock_1_times_stock_price_2_over_interest",
    "strike_1_over_stock_1_times_volatility_1_over_dividend_1",
    "strike_1_over_stock_1_times_interest_rate_over_dividend_1",
    "strike_1_over_stock_1_times_stock_price_1_over_dividend_1",
    "strike_1_over_stock_1_times_volatility_2_over_dividend_2",
    "strike_1_over_stock_1_times_interest_rate_over_dividend_2",
    "strike_1_over_stock_1_times_stock_price_2_over_dividend_2",
    "strike_1_over_stock_1_times_interest_rate_over_volatility_1",
    "strike_1_over_stock_1_times_dividend_yield_1_over_volatility_1",
    "strike_1_over_stock_1_times_stock_price_1_over_volatility_1",
    "strike_1_over_stock_1_times_interest_rate_over_volatility_2",
    "strike_1_over_stock_1_times_dividend_yield_2_over_volatility_2",
    "strike_1_over_stock_1_times_stock_price_2_over_volatility_2",
    "strike_2_over_stock_2_times_volatility_1_over_interest",
    "strike_2_over_stock_2_times_volatility_2_over_interest",
    "strike_2_over_stock_2_times_dividend_yield_1_over_interest",
    "strike_2_over_stock_2_times_dividend_yield_2_over_interest",
    "strike_2_over_stock_2_times_stock_price_1_over_interest",
    "strike_2_over_stock_2_times_stock_price_2_over_interest",
    "strike_2_over_stock_2_times_volatility_1_over_dividend_1",
    "strike_2_over_stock_2_times_interest_rate_over_dividend_1",
    "strike_2_over_stock_2_times_stock_price_1_over_dividend_1",
    "strike_2_over_stock_2_times_volatility_2_over_dividend_2",
    "strike_2_over_stock_2_times_interest_rate_over_dividend_2",
    "strike_2_over_stock_2_times_stock_price_2_over_dividend_2",
    "strike_2_over_stock_2_times_interest_rate_over_volatility_1",
    "strike_2_over_stock_2_times_dividend_yield_1_over_volatility_1",
    "strike_2_over_stock_2_times_stock_price_1_over_volatility_1",
    "strike_2_over_stock_2_times_interest_rate_over_volatility_2",
    "strike_2_over_stock_2_times_dividend_yield_2_over_volatility_2",
    "strike_2_over_stock_2_times_stock_price_2_over_volatility_2",
    "volatility_1_over_interest_times_volatility_2_over_interest",
    "volatility_1_over_interest_times_dividend_yield_1_over_interest",
    "volatility_1_over_interest_times_dividend_yield_2_over_interest",
    "volatility_1_over_interest_times_stock_price_1_over_interest",
    "volatility_1_over_interest_times_stock_price_2_over_interest",
    "volatility_1_over_interest_times_volatility_1_over_dividend_1",
    "volatility_1_over_interest_times_interest_rate_over_dividend_1",
    "volatility_1_over_interest_times_stock_price_1_over_dividend_1",
    "volatility_1_over_interest_times_volatility_2_over_dividend_2",
    "volatility_1_over_interest_times_interest_rate_over_dividend_2",
    "volatility_1_over_interest_times_stock_price_2_over_dividend_2",
    "volatility_1_over_interest_times_interest_rate_over_volatility_1",
    "volatility_1_over_interest_times_dividend_yield_1_over_volatility_1",
    "volatility_1_over_interest_times_stock_price_1_over_volatility_1",
    "volatility_1_over_interest_times_interest_rate_over_volatility_2",
    "volatility_1_over_interest_times_dividend_yield_2_over_volatility_2",
    "volatility_1_over_interest_times_stock_price_2_over_volatility_2",
    "volatility_2_over_interest_times_dividend_yield_1_over_interest",
    "volatility_2_over_interest_times_dividend_yield_2_over_interest",
    "volatility_2_over_interest_times_stock_price_1_over_interest",
    "volatility_2_over_interest_times_stock_price_2_over_interest",
    "volatility_2_over_interest_times_volatility_1_over_dividend_1",
    "volatility_2_over_interest_times_interest_rate_over_dividend_1",
    "volatility_2_over_interest_times_stock_price_1_over_dividend_1",
    "volatility_2_over_interest_times_volatility_2_over_dividend_2",
    "volatility_2_over_interest_times_interest_rate_over_dividend_2",
    "volatility_2_over_interest_times_stock_price_2_over_dividend_2",
    "volatility_2_over_interest_times_interest_rate_over_volatility_1",
    "volatility_2_over_interest_times_dividend_yield_1_over_volatility_1",
    "volatility_2_over_interest_times_stock_price_1_over_volatility_1",
    "volatility_2_over_interest_times_interest_rate_over_volatility_2",
    "volatility_2_over_interest_times_dividend_yield_2_over_volatility_2",
    "volatility_2_over_interest_times_stock_price_2_over_volatility_2",
    "dividend_yield_1_over_interest_times_dividend_yield_2_over_interest",
    "dividend_yield_1_over_interest_times_stock_price_1_over_interest",
    "dividend_yield_1_over_interest_times_stock_price_2_over_interest",
    "dividend_yield_1_over_interest_times_volatility_1_over_dividend_1",
    "dividend_yield_1_over_interest_times_interest_rate_over_dividend_1",
    "dividend_yield_1_over_interest_times_stock_price_1_over_dividend_1",
    "dividend_yield_1_over_interest_times_volatility_2_over_dividend_2",
    "dividend_yield_1_over_interest_times_interest_rate_over_dividend_2",
    "dividend_yield_1_over_interest_times_stock_price_2_over_dividend_2",
    "dividend_yield_1_over_interest_times_interest_rate_over_volatility_1",
    "dividend_yield_1_over_interest_times_dividend_yield_1_over_volatility_1",
    "dividend_yield_1_over_interest_times_stock_price_1_over_volatility_1",
    "dividend_yield_1_over_interest_times_interest_rate_over_volatility_2",
    "dividend_yield_1_over_interest_times_dividend_yield_2_over_volatility_2",
    "dividend_yield_1_over_interest_times_stock_price_2_over_volatility_2",
    "dividend_yield_2_over_interest_times_stock_price_1_over_interest",
    "dividend_yield_2_over_interest_times_stock_price_2_over_interest",
    "dividend_yield_2_over_interest_times_volatility_1_over_dividend_1",
    "dividend_yield_2_over_interest_times_interest_rate_over_dividend_1",
    "dividend_yield_2_over_interest_times_stock_price_1_over_dividend_1",
    "dividend_yield_2_over_interest_times_volatility_2_over_dividend_2",
    "dividend_yield_2_over_interest_times_interest_rate_over_dividend_2",
    "dividend_yield_2_over_interest_times_stock_price_2_over_dividend_2",
    "dividend_yield_2_over_interest_times_interest_rate_over_volatility_1",
    "dividend_yield_2_over_interest_times_dividend_yield_1_over_volatility_1",
    "dividend_yield_2_over_interest_times_stock_price_1_over_volatility_1",
    "dividend_yield_2_over_interest_times_interest_rate_over_volatility_2",
    "dividend_yield_2_over_interest_times_dividend_yield_2_over_volatility_2",
    "dividend_yield_2_over_interest_times_stock_price_2_over_volatility_2",
    "stock_price_1_over_interest_times_stock_price_2_over_interest",
    "stock_price_1_over_interest_times_volatility_1_over_dividend_1",
    "stock_price_1_over_interest_times_interest_rate_over_dividend_1",
    "stock_price_1_over_interest_times_stock_price_1_over_dividend_1",
    "stock_price_1_over_interest_times_volatility_2_over_dividend_2",
    "stock_price_1_over_interest_times_interest_rate_over_dividend_2",
    "stock_price_1_over_interest_times_stock_price_2_over_dividend_2",
    "stock_price_1_over_interest_times_interest_rate_over_volatility_1",
    "stock_price_1_over_interest_times_dividend_yield_1_over_volatility_1",
    "stock_price_1_over_interest_times_stock_price_1_over_volatility_1",
    "stock_price_1_over_interest_times_interest_rate_over_volatility_2",
    "stock_price_1_over_interest_times_dividend_yield_2_over_volatility_2",
    "stock_price_1_over_interest_times_stock_price_2_over_volatility_2",
    "stock_price_2_over_interest_times_volatility_1_over_dividend_1",
    "stock_price_2_over_interest_times_interest_rate_over_dividend_1",
    "stock_price_2_over_interest_times_stock_price_1_over_dividend_1",
    "stock_price_2_over_interest_times_volatility_2_over_dividend_2",
    "stock_price_2_over_interest_times_interest_rate_over_dividend_2",
    "stock_price_2_over_interest_times_stock_price_2_over_dividend_2",
    "stock_price_2_over_interest_times_interest_rate_over_volatility_1",
    "stock_price_2_over_interest_times_dividend_yield_1_over_volatility_1",
    "stock_price_2_over_interest_times_stock_price_1_over_volatility_1",
    "stock_price_2_over_interest_times_interest_rate_over_volatility_2",
    "stock_price_2_over_interest_times_dividend_yield_2_over_volatility_2",
    "stock_price_2_over_interest_times_stock_price_2_over_volatility_2",
    "volatility_1_over_dividend_1_times_interest_rate_over_dividend_1",
    "volatility_1_over_dividend_1_times_stock_price_1_over_dividend_1",
    "volatility_1_over_dividend_1_times_volatility_2_over_dividend_2",
    "volatility_1_over_dividend_1_times_interest_rate_over_dividend_2",
    "volatility_1_over_dividend_1_times_stock_price_2_over_dividend_2",
    "volatility_1_over_dividend_1_times_interest_rate_over_volatility_1",
    "volatility_1_over_dividend_1_times_dividend_yield_1_over_volatility_1",
    "volatility_1_over_dividend_1_times_stock_price_1_over_volatility_1",
    "volatility_1_over_dividend_1_times_interest_rate_over_volatility_2",
    "volatility_1_over_dividend_1_times_dividend_yield_2_over_volatility_2",
    "volatility_1_over_dividend_1_times_stock_price_2_over_volatility_2",
    "interest_rate_over_dividend_1_times_stock_price_1_over_dividend_1",
    "interest_rate_over_dividend_1_times_volatility_2_over_dividend_2",
    "interest_rate_over_dividend_1_times_interest_rate_over_dividend_2",
    "interest_rate_over_dividend_1_times_stock_price_2_over_dividend_2",
    "interest_rate_over_dividend_1_times_interest_rate_over_volatility_1",
    "interest_rate_over_dividend_1_times_dividend_yield_1_over_volatility_1",
    "interest_rate_over_dividend_1_times_stock_price_1_over_volatility_1",
    "interest_rate_over_dividend_1_times_interest_rate_over_volatility_2",
    "interest_rate_over_dividend_1_times_dividend_yield_2_over_volatility_2",
    "interest_rate_over_dividend_1_times_stock_price_2_over_volatility_2",
    "stock_price_1_over_dividend_1_times_volatility_2_over_dividend_2",
    "stock_price_1_over_dividend_1_times_interest_rate_over_dividend_2",
    "stock_price_1_over_dividend_1_times_stock_price_2_over_dividend_2",
    "stock_price_1_over_dividend_1_times_interest_rate_over_volatility_1",
    "stock_price_1_over_dividend_1_times_dividend_yield_1_over_volatility_1",
    "stock_price_1_over_dividend_1_times_stock_price_1_over_volatility_1",
    "stock_price_1_over_dividend_1_times_interest_rate_over_volatility_2",
    "stock_price_1_over_dividend_1_times_dividend_yield_2_over_volatility_2",
    "stock_price_1_over_dividend_1_times_stock_price_2_over_volatility_2",
    "volatility_2_over_dividend_2_times_interest_rate_over_dividend_2",
    "volatility_2_over_dividend_2_times_stock_price_2_over_dividend_2",
    "volatility_2_over_dividend_2_times_interest_rate_over_volatility_1",
    "volatility_2_over_dividend_2_times_dividend_yield_1_over_volatility_1",
    "volatility_2_over_dividend_2_times_stock_price_1_over_volatility_1",
    "volatility_2_over_dividend_2_times_interest_rate_over_volatility_2",
    "volatility_2_over_dividend_2_times_dividend_yield_2_over_volatility_2",
    "volatility_2_over_dividend_2_times_stock_price_2_over_volatility_2",
    "interest_rate_over_dividend_2_times_stock_price_2_over_dividend_2",
    "interest_rate_over_dividend_2_times_interest_rate_over_volatility_1",
    "interest_rate_over_dividend_2_times_dividend_yield_1_over_volatility_1",
    "interest_rate_over_dividend_2_times_stock_price_1_over_volatility_1",
    "interest_rate_over_dividend_2_times_interest_rate_over_volatility_2",
    "interest_rate_over_dividend_2_times_dividend_yield_2_over_volatility_2",
    "interest_rate_over_dividend_2_times_stock_price_2_over_volatility_2",
    "stock_price_2_over_dividend_2_times_interest_rate_over_volatility_1",
    "stock_price_2_over_dividend_2_times_dividend_yield_1_over_volatility_1",
    "stock_price_2_over_dividend_2_times_stock_price_1_over_volatility_1",
    "stock_price_2_over_dividend_2_times_interest_rate_over_volatility_2",
    "stock_price_2_over_dividend_2_times_dividend_yield_2_over_volatility_2",
    "stock_price_2_over_dividend_2_times_stock_price_2_over_volatility_2",
    "interest_rate_over_volatility_1_times_dividend_yield_1_over_volatility_1",
    "interest_rate_over_volatility_1_times_stock_price_1_over_volatility_1",
    "interest_rate_over_volatility_1_times_interest_rate_over_volatility_2",
    "interest_rate_over_volatility_1_times_dividend_yield_2_over_volatility_2",
    "interest_rate_over_volatility_1_times_stock_price_2_over_volatility_2",
    "dividend_yield_1_over_volatility_1_times_stock_price_1_over_volatility_1",
    "dividend_yield_1_over_volatility_1_times_interest_rate_over_volatility_2",
    "dividend_yield_1_over_volatility_1_times_dividend_yield_2_over_volatility_2",
    "dividend_yield_1_over_volatility_1_times_stock_price_2_over_volatility_2",
    "stock_price_1_over_volatility_1_times_interest_rate_over_volatility_2",
    "stock_price_1_over_volatility_1_times_dividend_yield_2_over_volatility_2",
    "stock_price_1_over_volatility_1_times_stock_price_2_over_volatility_2",
    "interest_rate_over_volatility_2_times_dividend_yield_2_over_volatility_2",
    "interest_rate_over_volatility_2_times_stock_price_2_over_volatility_2",
    "dividend_yield_2_over_volatility_2_times_stock_price_2_over_volatility_2",
    "option_type_put",
]

In [43]:
(len(out_features) * 1000000 * 8) / (1024 * 1024 * 1024)

5.81890344619751

In [24]:
lengths

[2, 51, 1, 10, 5, 5, 6]

In [15]:
import pandas as pd

In [16]:
training_data = pd.read_csv("../data/03_splitted/Worst_Off/training_data.csv")
test_data = pd.read_csv("../data/03_splitted/Worst_Off/testing_data.csv")

In [17]:
from sklearn.decomposition import PCA

In [18]:
X_train = training_data.iloc[:, :-1]
y_train = training_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

In [19]:
y_train

0         0.0000
1        37.9436
2         0.0548
3         8.3174
4         2.6999
          ...   
73478     0.0307
73479     2.7448
73480     5.2139
73481    19.5519
73482     3.3569
Name: price, Length: 73483, dtype: float64

In [ ]:
df = pd.read_csv("../data/02_processed/Worst_Off_processed_dataset.csv")

In [ ]:
df

In [22]:
corr = df.corr()

In [26]:
x = corr["price"].sort_values(ascending=False)

In [28]:
x[x>0.1]

price                                               1.000000
volatility_2_times_log1p_T                          0.510986
volatility_1_times_log1p_T                          0.510255
volatility_2_times_sqrt_T                           0.509130
volatility_1_times_sqrt_T                           0.508229
                                                      ...   
interest_rate_times_volatility_2_over_dividend_2    0.111578
volatility_2_times_interest_rate_over_dividend_1    0.110532
volatility_1_times_interest_rate_over_dividend_1    0.108626
interest_rate_times_volatility_1_over_dividend_1    0.108626
volatility_1_times_interest_rate_over_dividend_2    0.107971
Name: price, Length: 347, dtype: float64

In [29]:
pca_full = PCA(random_state=2025)
pca_full.fit(df.drop(columns='price'))

PCA(random_state=2025)

In [9]:
import numpy as np

In [30]:
print("Model Trainer: ", "PCA for full dataset calculated")

# Adding the amount of variance explained by each component
cum_var = np.cumsum(pca_full.explained_variance_ratio_)
print(cum_var)

Model Trainer:  PCA for full dataset calculated
[0.99999999 0.99999999 0.99999999 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.     

In [ ]:


# Setting a threshold for the amount of variance to be explained
# Here I just tried different values since 95 till 99 were in 20-ish range
# I decided to go with 98%, not too crazy and not too low
Var_threshold = 0.98
Number_of_components = np.where(cum_var > Var_threshold)[0][
    0
]  # this the index of the first component that explains more than 98% of the variance

print("Model Trainer: ", f"{Number_of_components} components selected (98% var)")

pca = PCA(n_components=Number_of_components, random_state=2025)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [34]:
df

,Option_type,S,K,T,r,sigma,q,bs_price,mc_price
0,call,50.0,100,0.08,0.01,0.1,0.00,1.026955e-133,0.000000
1,call,50.0,100,0.08,0.01,0.1,0.01,5.122990e-134,0.000000
2,call,50.0,100,0.08,0.01,0.1,0.02,2.553578e-134,0.000000
3,call,50.0,100,0.08,0.01,0.1,0.03,1.271828e-134,0.000000
4,call,50.0,100,0.08,0.01,0.1,0.04,6.329384e-135,0.000000
...,...,...,...,...,...,...,...,...,...
152995,put,150.0,100,5.00,0.10,0.4,0.05,9.607684e+00,9.582241
152996,put,150.0,100,5.00,0.10,0.8,0.03,2.920857e+01,29.189305
152997,put,150.0,100,5.00,0.10,0.8,0.02,2.860204e+01,28.554906
152998,put,150.0,100,5.00,0.10,0.8,0.05,3.042100e+01,30.509238


In [35]:
df.round(4)

,Option_type,S,K,T,r,sigma,q,bs_price,mc_price
0,call,50.0,100,0.08,0.01,0.1,0.00,0.0000,0.0000
1,call,50.0,100,0.08,0.01,0.1,0.01,0.0000,0.0000
2,call,50.0,100,0.08,0.01,0.1,0.02,0.0000,0.0000
3,call,50.0,100,0.08,0.01,0.1,0.03,0.0000,0.0000
4,call,50.0,100,0.08,0.01,0.1,0.04,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
152995,put,150.0,100,5.00,0.10,0.4,0.05,9.6077,9.5822
152996,put,150.0,100,5.00,0.10,0.8,0.03,29.2086,29.1893
152997,put,150.0,100,5.00,0.10,0.8,0.02,28.6020,28.5549
152998,put,150.0,100,5.00,0.10,0.8,0.05,30.4210,30.5092


# EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("../data/options_dataset.csv")

In [3]:
df.describe()

,S,K,T,r,sigma,q,bs_price,mc_price
count,153000.000000,153000.0,153000.000000,153000.00000,153000.000000,153000.000000,1.530000e+05,153000.000000
mean,100.000000,100.0,1.983000,0.05200,0.420000,0.025000,2.340649e+01,23.406672
std,29.439299,0.0,1.565125,0.03265,0.256126,0.017078,1.971527e+01,19.716090
min,50.000000,100.0,0.080000,0.01000,0.100000,0.000000,3.147372e-135,0.000000
25%,74.000000,100.0,0.500000,0.02500,0.200000,0.010000,5.232921e+00,5.232654
50%,100.000000,100.0,1.750000,0.05000,0.400000,0.025000,2.066067e+01,20.664333
75%,126.000000,100.0,3.000000,0.07500,0.600000,0.040000,3.746151e+01,37.460397
max,150.000000,100.0,5.000000,0.10000,0.800000,0.050000,1.167373e+02,114.659695


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153000 entries, 0 to 152999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Option_type  153000 non-null  object 
 1   S            153000 non-null  float64
 2   K            153000 non-null  int64  
 3   T            153000 non-null  float64
 4   r            153000 non-null  float64
 5   sigma        153000 non-null  float64
 6   q            153000 non-null  float64
 7   bs_price     153000 non-null  float64
 8   mc_price     153000 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 10.5+ MB


In [ ]:
fig = plt.figure(figsize=(40, 40))

In [38]:
sns.pairplot(df, hue="Option_type", diag_kind="kde", height=2.5)

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x0000019B64446290> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt

